<a href="https://colab.research.google.com/github/hshen13/workable-colab/blob/main/AI_in_math_equational_theories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 克隆 equational_theories 项目代码到 Colab 虚拟机
!git clone https://github.com/teorth/equational_theories.git

# 查看下载的文件结构（可选）
!find equational_theories -maxdepth=2 | sort


Cloning into 'equational_theories'...
remote: Enumerating objects: 18248, done.
remote: Counting objects: 100% (825/825), done.
remote: Compressing objects: 100% (325/325), done.
remote: Total 18248 (delta 722), reused 500 (delta 499), pack-reused 17423 (from 3)
Receiving objects: 100% (18248/18248), 60.65 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (13909/13909), done.
Updating files: 100% (1717/1717), done.
find: unknown predicate `-maxdepth=2'


In [ ]:
# 定义检测左单位元和右单位元性质的函数
# 对于一个 magma，其操作表用一个嵌套列表表示，
# 假设元素为 0, 1, 2, …, n-1，table[i][j] 表示 i ◇ j 的结果。

def has_left_identity(table, candidate):
    """
    检查 candidate 是否为左单位元，即对于所有 x, table[candidate][x] == x
    """
    n = len(table)
    for x in range(n):
        if table[candidate][x] != x:
            return False
    return True

def has_right_identity(table, candidate):
    """
    检查 candidate 是否为右单位元，即对于所有 x, table[x][candidate] == x
    """
    n = len(table)
    for x in range(n):
        if table[x][candidate] != x:
            return False
    return True

# 定义一个 2 元 magma 的操作表
# 我们希望 e = 0 为左单位元：
#   table[0] = [0, 1]  意味着 0 ◇ 0 = 0, 0 ◇ 1 = 1
# 但在第二行，table[1] = [0, 1]，即 1 ◇ 0 = 0 ≠ 1，所以 0 不是右单位元
table = [
    [0, 1],
    [0, 1]
]

print("Magma 操作表：")
for row in table:
    print(row)

# 检查每个候选元素是否为左单位元和右单位元
n = len(table)
for candidate in range(n):
    left = has_left_identity(table, candidate)
    right = has_right_identity(table, candidate)
    print(f"候选元素 {candidate}：左单位元？ {left}，右单位元？ {right}")


Magma 操作表：
[0, 1]
[0, 1]
候选元素 0：左单位元？ True，右单位元？ False
候选元素 1：左单位元？ True，右单位元？ False


In [ ]:
# 示例：运行项目中的脚本进行测试（具体参数需参考项目文档），验证或者直接证伪某个implication关系
!python3 equational_theories/scripts/explore_magma.py "[[0,1],[0,1]]" --ids 1,2 --verbose


Magma table: [[0, 1], [0, 1]]

|       | **0** | **1** |
|-------|-------|-------|
| **0** |   0   |   1   |
| **1** |   0   |   1   |

```
0 ◇ 0 = 0     0 ◇ 1 = 1
1 ◇ 0 = 0     1 ◇ 1 = 1
```

```
    1. x = x ✅
       0 = 0 # example with x=0

    2. x = y ❌
       0 ≠ 1 # counterexample with x=0, y=1

```

Magma tested against 2 equations:
* 1 passed (50.0%)
* 1 failed (50.0%)


In [ ]:
#找反例的例子
import random
import itertools

# 定义集合大小
n = 5  # magma 元素集合 {0,1,2,3,4}

def generate_random_magma(n):
    """
    随机生成一个 n x n 的 magma 运算表，
    每个运算结果为 0 到 n-1 的随机整数。
    """
    return [[random.randrange(n) for _ in range(n)] for _ in range(n)]

def check_eq_A(table):
    """
    检查候选等式 A 是否成立：

    ((x ◇ y) ◇ z) ◇ (u ◇ v) = (x ◇ y) ◇ (z ◇ (u ◇ v))

    对于 table 表示的 magma，枚举所有 5 个变量的赋值（x,y,z,u,v）。
    """
    for x, y, z, u, v in itertools.product(range(n), repeat=5):
        # 计算左侧：((x ◇ y) ◇ z) ◇ (u ◇ v)
        a = table[x][y]        # x ◇ y
        b = table[a][z]        # (x ◇ y) ◇ z
        c = table[u][v]        # u ◇ v
        left = table[b][c]     # ((x ◇ y) ◇ z) ◇ (u ◇ v)

        # 计算右侧：(x ◇ y) ◇ (z ◇ (u ◇ v))
        a = table[x][y]        # x ◇ y
        d = table[u][v]        # u ◇ v
        e = table[z][d]        # z ◇ (u ◇ v)
        right = table[a][e]    # (x ◇ y) ◇ (z ◇ (u ◇ v))

        if left != right:
            return False
    return True

def check_eq_B(table):
    """
    检查候选等式 B 是否成立：

    (x ◇ y) ◇ (z ◇ (u ◇ v)) = (x ◇ (y ◇ z)) ◇ (u ◇ v)

    同样枚举所有 5 个变量的赋值验证此等式。
    """
    for x, y, z, u, v in itertools.product(range(n), repeat=5):
        # 计算左侧： (x ◇ y) ◇ (z ◇ (u ◇ v))
        a = table[x][y]        # x ◇ y
        d = table[u][v]        # u ◇ v
        b = table[z][d]        # z ◇ (u ◇ v)
        left = table[a][b]     # (x ◇ y) ◇ (z ◇ (u ◇ v))

        # 计算右侧： (x ◇ (y ◇ z)) ◇ (u ◇ v)
        f = table[y][z]        # y ◇ z
        c = table[x][f]        # x ◇ (y ◇ z)
        right = table[c][d]    # (x ◇ (y ◇ z)) ◇ (u ◇ v)

        if left != right:
            return False
    return True

# 测试蕴涵关系：寻找一个 magma 满足候选等式 A 但不满足候选等式 B

num_samples = 1000  # 随机测试样本数
counterexample_found = False
counterexample = None

for i in range(num_samples):
    table = generate_random_magma(n)
    if check_eq_A(table):  # 如果满足候选等式 A
        if not check_eq_B(table):  # 但不满足候选等式 B
            counterexample_found = True
            counterexample = table
            print("在样本", i+1, "中找到反例！")
            break

if counterexample_found:
    print("\n找到满足候选等式 A 但不满足候选等式 B 的 magma 运算表：")
    for row in counterexample:
        print(row)
else:
    print("在", num_samples, "个随机样本中，没有找到反例。\n这并不证明 A 蕴涵 B，但在有限样本内没有发现反例。")


在 1000 个随机样本中，没有找到反例。
这并不证明 A 蕴涵 B，但在有限样本内没有发现反例。
